In [ ]:
import looker_sdk
import pandas as pd
import os
from tqdm import tqdm
from datetime import datetime


In [ ]:
# os.environ["LOOKERSDK_BASE_URL"] = "LOOKERSDK_BASE_URL" #If your looker URL has .cloud in it (hosted on GCP), do not include :19999 (ie: https://your.cloud.looker.com).
# os.environ["LOOKERSDK_API_VERSION"] = "4.0" #As of Looker v23.18+, the 3.0 and 3.1 versions of the API are removed. Use "4.0" here.
# os.environ["LOOKERSDK_VERIFY_SSL"] = "true" #Defaults to true if not set. SSL verification should generally be on unless you have a real good reason not to use it. Valid options: true, y, t, yes, 1.
# os.environ["LOOKERSDK_TIMEOUT"] = "120" #Seconds till request timeout. Standard default is 120.

# os.environ["LOOKERSDK_CLIENT_ID"] =  "LOOKERSDK_CLIENT_ID" #No defaults.
# os.environ["LOOKERSDK_CLIENT_SECRET"] = <hidden> #No defaults. This should be protected at all costs.
if len(os.environ["LOOKERSDK_CLIENT_ID"]) >0:
    print("All environment variables set.")
else:
    print('error, variables not set!!')

All environment variables set.


In [3]:
sdk = looker_sdk.init40()
print('Looker SDK 4.0 initialized successfully.')

Looker SDK 4.0 initialized successfully.


In [4]:
# check if connection works fine
my_user = sdk.me()

# print(my_user.first_name) # Model dot notation
print(my_user["first_name"]) # Dictionary

Moataz


In [64]:
looks_ids = [l.id for l in sdk.all_looks()] 
dbd_ids=[d.id for d in sdk.all_dashboards()] 
dbd_ids=['125'] 

print(f'There are {len(looks_ids)} looks and {len(dbd_ids)} dashboards.')


There are 82 looks and 1 dashboards.


In [83]:
# uses series_labels for all
data1 = [] # all main columns
data2 = [] # relabeled names (ie Company Name -> Co Name)
data3 = [] # filtered values 

titles=[] # dashboard titles
for i,id in tqdm(enumerate(dbd_ids), total=len(dbd_ids)):
    dbd=sdk.dashboard(id)
    dbd_title=dbd['title'] # dashboard title
    if dbd_title !='Reimbursement Class Post Migration Validation':
        continue
    titles.append(dbd_title)
    # print(f'{i} {dbd_title} ---',end=' ')
    # if (i)%10 ==0:
    #     print(f'{i} {dbd_title} ---')
        # print('')  
    dbd_created_at=dbd['created_at']
    dbd_updated_at=dbd['updated_at']
    dbd_last_updater_name=dbd['last_updater_name']
    dbd_last_viewed_at=dbd['last_viewed_at']
    dbd_deleted=dbd['deleted']
    #---
    dbd_els=dbd['dashboard_elements'] # dashboard elements (tables, visuals, etc)
    for tbl in dbd_els:
        table_title=tbl['title']
        print(table_title)
        if tbl.result_maker is None:
            print(f'0 no result_maker for [{dbd_title}].[{table_title}]')
            continue
        if tbl.result_maker.query is None:
            print(f'0.5 no query for [{dbd_title}].[{table_title}]')
            continue
        vis_config=tbl.result_maker.query.vis_config # ideally if the element is a table, this will have a value
        if 'series_labels' not in vis_config:
            print(f'2 no series_labels for [{dbd_title}].[{table_title}]')
            print(tbl)
            break
            continue
        for k,v in vis_config['series_labels'].items():
            # print(' '.join(i), len(' '.join(i)))
            col_name= k # ledger_report_hra_hub.company_name
            col_label= v # Co Name (renamed column)
            d1={
                "dashboard_id": id,
                "dashboard_title": dbd_title,
                "is_deleted": dbd_deleted,
                "created_at": dbd_created_at,
                "updated_at": dbd_updated_at,
                "updated_by": dbd_last_updater_name,
                "viewed_at": dbd_last_viewed_at,
                "element_title": table_title,
                "col_name": col_name,
                "col_label": col_label
            }
            print(d1)
            data1.append(d1)
            
        for k,v in vis_config['series_labels'].items():
            # print(' '.join(i), len(' '.join(i)))
            col_name2= k # ledger_report_hra_hub.company_name
            col_label2= v # Co Name (renamed column)
            data2.append({
                "dashboard_id": id,
                "col_name": col_name2,
                "col_label": col_label2
            })
    dbd_filters=dbd['dashboard_filters']
    for filter in dbd_filters:
        filter_title=filter['title'] # how does it look like in the dashboard
        col_name3=filter['dimension'] # what field is this filter connected to?
        data3.append({
                "dashboard_id": id,
                "col_name": col_name3,
                "filter": filter_title
            })
        
            
df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2)
df3 = pd.DataFrame(data3)

100%|██████████| 1/1 [00:00<00:00,  2.04it/s]

Reimbursement Classes per Company
{'dashboard_id': '125', 'dashboard_title': 'Reimbursement Class Post Migration Validation', 'is_deleted': False, 'created_at': datetime.datetime(2024, 8, 15, 4, 7, 25, tzinfo=datetime.timezone.utc), 'updated_at': datetime.datetime(2024, 10, 25, 2, 40, 23, tzinfo=datetime.timezone.utc), 'updated_by': 'Moataz Imad', 'viewed_at': datetime.datetime(2024, 12, 19, 21, 30, 45, tzinfo=datetime.timezone.utc), 'element_title': 'Reimbursement Classes per Company', 'col_name': 'company_rc_validation.m_group', 'col_label': 'Migration Group'}
{'dashboard_id': '125', 'dashboard_title': 'Reimbursement Class Post Migration Validation', 'is_deleted': False, 'created_at': datetime.datetime(2024, 8, 15, 4, 7, 25, tzinfo=datetime.timezone.utc), 'updated_at': datetime.datetime(2024, 10, 25, 2, 40, 23, tzinfo=datetime.timezone.utc), 'updated_by': 'Moataz Imad', 'viewed_at': datetime.datetime(2024, 12, 19, 21, 30, 45, tzinfo=datetime.timezone.utc), 'element_title': 'Reimburse

In [20]:
print(f'df1: {df1.columns}')
print(f'df2: {df2.columns}')
print(f'df3: {df3.columns}')

df1: Index(['dashboard_id', 'dashboard_title', 'is_deleted', 'created_at',
       'updated_at', 'updated_by', 'viewed_at', 'element_title', 'col_name',
       'col_label'],
      dtype='object')
df2: Index(['dashboard_id', 'col_name', 'col_label'], dtype='object')
df3: Index(['dashboard_id', 'col_name', 'filter'], dtype='object')


In [33]:
# join df1, df2, df3 on dashboard_id, and column_name
df_merged = pd.merge(df1, df2, on=['dashboard_id', 'col_name'], how='left', suffixes=('', '_2'))

result = pd.merge(df_merged, df3, on=['dashboard_id', 'col_name'], how='left', suffixes=('', '_df3'))

result.to_csv('result.csv')

### Write to Bigquery

In [ ]:
result=pd.read_csv('result.csv',index_col=0)


In [ ]:
result['column_lable']=result.apply(lambda x: x['col_label'] if pd.isnull(x['col_label_2']) else x['col_label_2'], axis=1)
result['timestamp']=datetime.now()
final_df=result[['timestamp','dashboard_id', 'dashboard_title', 'is_deleted', 'created_at',
       'updated_at', 'updated_by', 'viewed_at', 'element_title', 'col_name','column_lable','filter']]

In [ ]:
print(1)
print(2)
print(3)
raise ValueError('test')
print(4)

1
2
3


ValueError: test

#### Connect to Big Query

In [55]:
from google.cloud import bigquery
client = bigquery.Client(project="key-chalice-340021")

#### Write Table `pandas -> bigquery`


In [56]:
table_id = 'key-chalice-340021.reports.looker_report'

job_config = bigquery.LoadJobConfig(
    # write_disposition=bigquery.WriteDisposition.WRITE_APPEND # append (default if no config is set)
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE # overwrite
)

# Load the DataFrame and overwrite the table
job = client.load_table_from_dataframe(final_df, table_id, job_config=job_config)
job.result()

LoadJob<project=key-chalice-340021, location=US, id=ca3e9779-866e-48de-aed4-a928b761048c>

In [57]:
final_df.columns

Index(['timestamp', 'dashboard_id', 'dashboard_title', 'is_deleted',
       'created_at', 'updated_at', 'updated_by', 'viewed_at', 'element_title',
       'col_name', 'column_lable', 'filter'],
      dtype='object')